In [ ]:
!pip install pandas

In [124]:
import json
import pandas as pd
import re
import numpy as np

In [196]:
data = []
with open("long_queries.jsonl", 'r', encoding='utf-8', errors='replace') as f:
    for line in f:
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Skipping invalid line: {e}")
            
df_queries = pd.DataFrame(data)
df_queries.head()

Skipping invalid line: Extra data: line 1 column 1618 (char 1617)


,name,employee_tenure,department,job_title,company_size,company_sector,tasks,long_queries,line_idx,task_idx,task
0,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[resolution support customer service legal iss...,0,0,NaN
1,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[client account payment issues legal billing d...,0,1,NaN
2,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[escalate high priority cases to senior manage...,0,2,NaN
3,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[Drafting a document update response for clien...,0,3,NaN
4,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...","[Change of schedule coordination, Scheduling c...",0,4,NaN


In [190]:
len(df_queries)

104548

In [197]:
group_cols = [
    'name', 'employee_tenure', 'department', 'job_title',
    'company_size', 'company_sector', 'tasks'
]
df_queries['tasks'] = df_queries['tasks'].apply(tuple)
# Assign a unique group ID starting from 0
df_queries['user_idx'] = df_queries.groupby(group_cols).ngroup()
df_queries['tasks'] = df_queries['tasks'].apply(list)
df_queries.head()


,name,employee_tenure,department,job_title,company_size,company_sector,tasks,long_queries,line_idx,task_idx,task,user_idx
0,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[resolution support customer service legal iss...,0,0,NaN,17
1,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[client account payment issues legal billing d...,0,1,NaN,17
2,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[escalate high priority cases to senior manage...,0,2,NaN,17
3,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[Drafting a document update response for clien...,0,3,NaN,17
4,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...","[Change of schedule coordination, Scheduling c...",0,4,NaN,17


In [116]:
for _, row in df_tasks.iterrows():
    if len(row['tasks']) != 10:
        print(row)

In [115]:
df_tasks = df_queries.copy()
df_tasks['tasks'] = df_tasks['tasks'].apply(tuple)
columns_to_keep = [
    'name', 
    'employee_tenure', 
    'department', 
    'job_title', 
    'company_size', 
    'company_sector', 
    'tasks',
    'user_idx'
]
df_tasks = df_tasks[columns_to_keep].drop_duplicates()
df_tasks['tasks'] = df_tasks['tasks'].apply(list)
df_tasks.head()

,name,employee_tenure,department,job_title,company_size,company_sector,tasks,user_idx
0,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",17
10,Elijah,experienced,Sales / Business Development,Senior Sales Professional,small,Education & Research Institutions,[Meet with potential client to discuss researc...,2841
20,Mia,new,Information Technology (IT),Web Developer,small,Energy & Utilities,[Designing a new website for the company's ene...,7658
30,Harper,experienced,Information Technology (IT),Cloud Computing Professional,small,Healthcare & Pharmaceuticals,"[Monitoring AWS EC2 instance performance, Trou...",4303
40,James,new,Information Technology (IT),Web Developer,small,Software & Technology,[Reviewing project documentation for upcoming ...,6086


In [125]:
expected = np.arange(len(df_tasks)) 
actual = df_tasks['user_idx'].values

is_continuous = np.array_equal(np.sort(actual), expected)
print("Is user_idx continuous from 0 to len(df)-1?:", is_continuous)

Is user_idx continuous from 0 to len(df)-1?: True


In [118]:
len(df_tasks)

10456

In [159]:
df_tasks[df_tasks['user_idx'] == 1]['tasks'].tolist()

[['Respond to all urgent customer inquiries via phone and email',
  'Resolve customer complaints regarding service disruptions',
  'Conduct a routine audit of customer feedback forms to identify areas for improvement',
  'Schedule a meeting with the marketing team to discuss new campaign strategy',
  'Create and send welcome packages to newly acquired customers',
  'Collaborate with internal teams to resolve technical issues',
  'Develop and maintain knowledge base articles on common customer queries',
  'Track and report key performance indicators (KPIs) for customer satisfaction',
  'Escalate complex customer complaints to senior support staff',
  'Research and implement new technologies to enhance customer experience']]

In [139]:
df_tasks.to_json('user_tasks.json', orient = 'records', lines = True)

In [169]:
# Load the JSON file
with open("task_tool_mapping_out.json", "r") as f:
    data = json.load(f)

# Convert to DataFrame
annotation1 = pd.DataFrame(data["task_tool_mappings"])

In [170]:
assert annotation1.groupby(["user_idx"]).size().nunique() == 1, "uneven task counts!"

AssertionError: uneven task counts!

In [186]:
size = annotation1.groupby('user_idx').count()
for _ , row in size.iterrows():
    if row['task'] != 10:
        print(row)

task_idx          9
task              9
tool_to_search    9
Name: 1558, dtype: int64
task_idx          6
task              6
tool_to_search    6
Name: 6626, dtype: int64
task_idx          9
task              9
tool_to_search    9
Name: 9440, dtype: int64


In [188]:
annotation1[annotation1['user_idx'] == 6626]

,user_idx,task_idx,task,tool_to_search
66808,6626,0,Reviewing and optimizing ad campaigns for impr...,Confluence
66809,6626,1,Creating a comprehensive report on campaign pe...,Google Sheets
66810,6626,2,Meeting with cross-functional team to discuss ...,Jira
66811,6626,3,Developing a social media strategy for new pro...,Miro
66812,6626,4,Designing eye-catching ad creatives for upcomi...,Google Slides
66813,6626,5,Analyzing website traffic patterns to identify...,Google Analytics (though not listed - might be...


In [142]:
df = pd.read_json('user_tasks.json', lines=True)

# keep the first row for each user_idx
dupes = df.duplicated(subset="user_idx", keep="first")
if dupes.any():
    print("⚠️  removing", dupes.sum(), "duplicate user rows")
df = df[~dupes].reset_index(drop=True)


In [143]:
bad = df[ df["tasks"].str.len() != 10 ]
assert bad.empty, f"rows with !=10 tasks: {bad['user_idx'].tolist()}"


In [80]:
annotation1.loc[
    (annotation1['user_idx'] == 0) & (annotation1['task_idx'] == 0),
    'tool_to_search'
]

10                Gmail
99966            GitHub
99996     Google Sheets
100066       Confluence
100086       Confluence
              ...      
104216       Confluence
104266    Google Sheets
104296       Confluence
104346       Confluence
104476       Confluence
Name: tool_to_search, Length: 114, dtype: object

In [71]:
user_keys = ['name', 'employee_tenure', 'department', 'job_title', 'company_size', 'company_sector', 'tasks']
df_tasks['tasks'] = df_tasks['tasks'].apply(tuple)
df_queries['tasks'] = df_queries['tasks'].apply(tuple)
df_queries = df_queries.merge(
    df_tasks[user_keys + ['user_idx']],
    on=user_keys,
    how='left'
)
df_tasks['tasks'] = df_tasks['tasks'].apply(list)
df_queries['tasks'] = df_queries['tasks'].apply(list)
# Now df_queries has user_idx added
df_queries.head()

,name,employee_tenure,department,job_title,company_size,company_sector,tasks,long_queries,line_idx,task_idx,task,user_idx
0,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[resolution support customer service legal iss...,0,0,NaN,0
1,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[client account payment issues legal billing d...,0,1,NaN,0
2,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[escalate high priority cases to senior manage...,0,2,NaN,0
3,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[Drafting a document update response for clien...,0,3,NaN,0
4,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...","[Change of schedule coordination, Scheduling c...",0,4,NaN,0


In [75]:
a = pd.DataFrame(annotation1[['user_idx', 'task_idx']].value_counts())
a[a['count'] > 1]

count
user_idx task_idx       
1        0           334
         1           316
         6           295
         9           295
         8           295
...                  ...
41       4             2
         8             2
         7             2
         6             2
         5             2

[88 rows x 1 columns]

In [194]:
annotation1 = pd.read_json("annotation_1_out.json", lines=True)
annotation1.head()

,user_idx,task_idx,task,tool_to_search
0,2841,0,Meet with potential client to discuss research...,Confluence
1,2841,1,Develop and deliver sales presentation for new...,Google Slides
2,2841,2,Follow up on pending deals in pipeline,CRM Software
3,2841,3,Create sales forecast report for quarterly review,Google Sheets
4,2841,4,Collaborate with marketing team to create join...,Slack


In [201]:
annotation1[annotation1['user_idx'] == 17]

,user_idx,task_idx,task,tool_to_search
10,17,0,Respond to 3 customer complaints via email,Gmail
11,17,1,Investigate and resolve issue with client acco...,Confluence
12,17,2,Escalate 2 high-priority cases to senior manag...,Slack
13,17,3,Draft response to client's request for documen...,Google Doc
14,17,4,Coordinate with internal teams to facilitate s...,Slack
15,17,5,Manage and respond to incoming customer inquir...,Gmail
16,17,6,Process new client registration application,Google Sheets
17,17,7,Research and analyze data on competitor pricin...,Google Sheets
18,17,8,Develop and implement a solution plan for recu...,Confluence
19,17,9,Collaborate with colleagues to develop trainin...,Google Slides


In [198]:
keys = ['user_idx', 'task_idx']
df_queries = df_queries.merge(
    annotation1,
    on=keys,
    how='left'
)
df_queries.head()

,name,employee_tenure,department,job_title,company_size,company_sector,tasks,long_queries,line_idx,task_idx,task_x,user_idx,task_y,tool_to_search
0,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[resolution support customer service legal iss...,0,0,NaN,17,Respond to 3 customer complaints via email,Gmail
1,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[client account payment issues legal billing d...,0,1,NaN,17,Investigate and resolve issue with client acco...,Confluence
2,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[escalate high priority cases to senior manage...,0,2,NaN,17,Escalate 2 high-priority cases to senior manag...,Slack
3,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...",[Drafting a document update response for clien...,0,3,NaN,17,Draft response to client's request for documen...,Google Doc
4,Alexander,experienced,Customer Support / Service,Resolution Specialist,mid-sized,Legal & Law Firms,"[Respond to 3 customer complaints via email, I...","[Change of schedule coordination, Scheduling c...",0,4,NaN,17,Coordinate with internal teams to facilitate s...,Slack


In [199]:
df_queries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104548 entries, 0 to 104547
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   name             104548 non-null  object
 1   employee_tenure  104548 non-null  object
 2   department       104548 non-null  object
 3   job_title        104548 non-null  object
 4   company_size     104548 non-null  object
 5   company_sector   104548 non-null  object
 6   tasks            104548 non-null  object
 7   long_queries     104548 non-null  object
 8   line_idx         104548 non-null  int64 
 9   task_idx         104548 non-null  int64 
 10  task_x           103579 non-null  object
 11  user_idx         104548 non-null  int64 
 12  task_y           104548 non-null  object
 13  tool_to_search   104548 non-null  object
dtypes: int64(3), object(11)
memory usage: 11.2+ MB
